<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/S101lemDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing pipeline for DAAD project
## Stage 1: Lemmatization (German)

Example: Universal declaration of human rights

In [ ]:
# Downloading UDHR
!wget https://unicode.org/udhr/assemblies/udhr_txt.zip

In [ ]:
# cleanup if the route directory not empty
!cd udhr/
!rm udhr*
!rm index.xml

In [ ]:
%%bash
mkdir udhr
mv udhr_txt.zip udhr
cd udhr/
unzip udhr_txt.zip

In [15]:
%%bash
# delete lines which are not translations in some files (hy)
# delete between lines $a and $b inclusive
a=9
b=21
awk -v m=$a -v n=$b 'm <= NR && NR <= n {next} {print}' < udhr/udhr_hye.txt >udhr/udhr_hye2.txt 

In [20]:
%%bash
# put paragraph tags
awk '{print "<p>\n"$0 ; print "</p>"}' /content/udhr/udhr_deu_1996.txt >/content/udhr/udhr_deu_1996_v02.txt

In [ ]:
%%bash
# installing TreeTagger (en, de, ka)
mkdir treetagger
cd treetagger
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/english.par.gz
sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper
# changing options: no-unknown, sgml, lemma
mv /content/treetagger/cmd/tree-tagger-english /content/tree-tagger-english0
awk '{ if (NR == 9) print "OPTIONS=\"-token -lemma -sgml -no-unknown\""; else print $0}' /content/tree-tagger-english0 > /content/treetagger/cmd/tree-tagger-english
chmod a+x ./treetagger/cmd/tree-tagger-english
# downloading German and Georgian 
wget https://heibox.uni-heidelberg.de/f/ec8226edebb64a359407/?dl=1
mv index.html?dl=1 /content/treetagger/lib/german-utf8.par
wget https://heibox.uni-heidelberg.de/f/9183090d2bdb41e09055/?dl=1
mv index.html?dl=1 /content/treetagger/lib/georgian.par
wget https://heibox.uni-heidelberg.de/f/9cafab0509d64ed1ac4b/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-georgian2
# German2 = -no-unknown 
# note: tree-tagger-german will not work, as parameter files have not been downloaded, only use tree-tagger-german2 with utf8 encoding
wget https://heibox.uni-heidelberg.de/f/acb9b8a2fa4f40e08f8a/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-german2
chmod a+x /content/treetagger/cmd/tree-tagger-georgian2
chmod a+x /content/treetagger/cmd/tree-tagger-german2

In [17]:
%%bash
# output directory
mkdir udhrTT


In [21]:
# analysing German file 
!./treetagger/cmd/tree-tagger-german2 /content/udhr/udhr_deu_1996_v02.txt >/content/udhrTT/udhr_deu_1996_vert.txt

	reading parameters ...
	tagging ...
2000	 finished.


In [28]:
# !awk -F '\t' '(NF==3){printf "%s ", $3; if(FNR % 10000 == 0){printf "\n"}}' < /content/udhrTT/udhr_deu_1996_vert.txt >/content/udhrTT/udhr_deu_1996_lem.txt

# if we need <p> tags for paragraphs:
# !awk -F '\t' '(NF==3){printf "%s ", $3}(NF!=3){printf "\n%s\n", $0}' < /content/udhrTT/udhr_deu_1996_vert.txt >/content/udhrTT/udhr_deu_1996_lem.txt
# if we do not need paragraph tags
!awk -F '\t' '(NF==3){printf "%s ", $3}(NF!=3){printf "\n"}' < /content/udhrTT/udhr_deu_1996_vert.txt >/content/udhrTT/udhr_deu_1996_lem.txt